# Running multiple calculations on a given model


## Aim

This notebook demonstrates how different types of tasks can be connected within a workflow and run a workflow on an external computer. As an example, we start from a structure, optimize its geometry, compute descriptors, then use a filtering function to split the resulting structures into training fles, run Quantum Espresso to get energies on an external system(scarf) and finally run training on the output files. The goal is to show how workflows can be setup to run on external systems.

### Setup

The initial setup is very similar to the other tutorials, such as `singlepoint.ipynb`, which goes into more detail about what each step is doing

Load the aiida profile and code:

In [ ]:
from aiida import load_profile
load_profile()

In [ ]:
from aiida_mlip.data.model import ModelData
uri = "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model"
model = ModelData.from_uri(uri, architecture="mace_mp")

We can load the computer and set the directory in which we want to store the data to. The work directory is a placeholder so change accordingly.

In [ ]:
from aiida.orm import load_code, load_computer

janus_code = load_code("janus@localhost")

We must now choose the calculations to perform:

In [ ]:
from aiida.plugins import CalculationFactory
geomoptCalc = CalculationFactory("mlip.opt")
descriptorsCalc = CalculationFactory("mlip.descriptors")
trainCalc = CalculationFactory("mlip.train")


Before setting up the work graph, we first configure the `Quantum Espresso (QE)` task by defining the code and input parameters. Since we need to run QE on multiple structures, we create multiple `PwCalculation` tasks dynamically within the same task using `get_current_graph()`. This allows us to run QE for each structure and return the corresponding `TrajectoryData` and parameters for each.

In [ ]:
from aiida_workgraph import task
from aiida_workgraph.manager import get_current_graph
from aiida.orm import StructureData, load_group, KpointsData, SinglefileData
from ase.io import iread
from pathlib import Path
import yaml
from aiida_quantumespresso.calculations.pw import PwCalculation
from sample_split import process_and_split_data


@task.graph(outputs = ["test_file", "train_file", "valid_file"])
def qe(**inputs):

    qe_code = load_code("qe@scarf")
    wg = get_current_graph()

    kpoints = KpointsData()
    kpoints.set_kpoints_mesh(inputs['kpoint_mesh'].value)

    pseudo_family = load_group('SSSP/1.3/PBE/efficiency')
    files = {"test_file": inputs['test_file'],"train_file":inputs['train_file'],"valid_file":inputs['valid_file']}

    for file_name, file in files.items():
        with file.as_path() as path:
            for i, structs in enumerate(iread(path, format="extxyz")):
                
                structure = StructureData(ase=structs)
                pseudos = pseudo_family.get_pseudos(structure=structure)

                ecutwfc, ecutrho = pseudo_family.get_recommended_cutoffs(
                    structure=structure,
                    unit='Ry',
                )

                pw_params = {
                    "CONTROL": {
                        "calculation": "scf",
                        'tprnfor': True,
                        'tstress': True,
                    },
                    "SYSTEM": {
                        "ecutwfc": ecutwfc,
                        "ecutrho": ecutrho,
                    },
                }

                qe_task = wg.add_task(
                    PwCalculation,
                    code = qe_code,
                    parameters= pw_params,
                    kpoints= kpoints,
                    pseudos= pseudos,
                    metadata= inputs["calc_metadata"].value,
                    structure= structure,
                )

                structfile = f"{file_name}.struct{i}"
                wg.update_ctx({structfile:{
                    "trajectory":qe_task.outputs.output_trajectory,
                    "parameters": qe_task.outputs.output_parameters
                }})

    return {
        "test_file": wg.ctx.test_file,
        "train_file": wg.ctx.train_file,
        "valid_file": wg.ctx.valid_file
    }    


In [ ]:
from aiida_mlip.data.config import JanusConfigfile
from aiida.orm import Dict, TrajectoryData
from ase.io import write
from ase import units

@task.calcfunction(outputs = ["JanusConfigfile"])
def create_train_file(**inputs):

    training_files = {}
    
    for file_name, structs in inputs.items():
        path = Path(f"mlip_{file_name}.extxyz")

        for stuct_out_params in structs.values():
            
            trajectory = stuct_out_params["trajectory"]

            fileStructure = trajectory.get_structure(index=0)
            fileAtoms = fileStructure.get_ase()

            stress = trajectory.arrays["stress"][0]
            converted_stress = stress * units.GPa
            fileAtoms.info["qe_stress"] = converted_stress

            fileAtoms.info["units"] = {"energy": "eV","forces": "ev/Ang","stress": "ev/Ang^3"}
            fileAtoms.set_array("qe_forces", trajectory.arrays["forces"][0])

            parameters = stuct_out_params["parameters"]
            fileParams = parameters.get_dict()
            fileAtoms.info["qe_energy"] = fileParams["energy"]
            write(path, fileAtoms, append=True)

        training_files[file_name] = str(path.resolve())

    with open("JanusConfigfile.yml", "a") as f:
        yaml.safe_dump(training_files, f, sort_keys=False)

    return{'JanusConfigfile': JanusConfigfile(str(Path("JanusConfigfile.yml").resolve()))}

For this task, we are using a task to run a pure python function. This is to demonstrate the flexibility of tasks and how you can run pure python functions on the workchain. This returns `SinglefileData` instances of the test, train and valid files.

In [ ]:
@task.calcfunction(outputs = ["test_file", "train_file", "valid_file"])
def create_aiida_files(**inputs):
     
    files = process_and_split_data(**inputs)

    return {
        "train_file": SinglefileData(files["train_file"]),
        "test_file": SinglefileData(files["test_file"]),
        "valid_file": SinglefileData(files["valid_file"])
    }

We initialize the inputs we want for all the calculations. These variables can be changed depending on the configuration you are running and whether you want to change any inputs.

In [ ]:
from aiida.orm import Str, Float, Bool, Int, List

calc_inputs = {
    "code": janus_code,
    "model": model,
    "arch": Str(model.architecture),
    "device": Str("cpu"),
    "metadata": {"options": {"resources": {"num_machines": 1}}},
}

goemopt_inputs = {
    "fmax": Float(0.1),
    "opt_cell_lengths": Bool(False),
    "opt_cell_fully": Bool(True),
}

split_task_inputs = {
    "config_types": Str(""),
    "prefix": Str(""),
    "scale": Float(1.0e5),
    "append_mode": Bool(False),
}

qe_inputs = {
    "calc_metadata":  Dict({
        "options": {
            "resources": {
                "num_machines": 1,
                'num_mpiprocs_per_machine': 32,
            },
            'max_wallclock_seconds': 3600,
            'queue_name': 'scarf',                  
            'qos': 'scarf',
            'environment_variables': {},
            'withmpi': True,                       
            'prepend_text': '''
            module purge
            module use /work4/scd/scarf562/eb-common/modules/all
            module load amd-modules
            module load QuantumESPRESSO/7.2-foss-2023a
            ''',
            'append_text': ''  
        }
    }),
    "kpoint_mesh": List([1,1,1])
}

In [ ]:
from aiida_workgraph import WorkGraph
from aiida.orm import StructureData
from ase.io import iread

initial_structure = "../structures/NaCl-traj.xyz"

with WorkGraph("QE Calculation Workgraph") as wg:

    final_structures = {}

    for i, struct in enumerate(iread(initial_structure)):

        structure = StructureData(ase=struct)
        geomopt_calc = wg.add_task(
            geomoptCalc,
            **calc_inputs,
            **goemopt_inputs,
            struct=structure,
        )
        
        descriptors_calc = wg.add_task(
            descriptorsCalc,
            **calc_inputs,
            struct=geomopt_calc.outputs.final_structure,
            calc_per_element=True,
        )

        final_structures[f"structs{i}"] = descriptors_calc.outputs.xyz_output

    split_task = wg.add_task(
        create_aiida_files, 
        **split_task_inputs,
        trajectory_data=final_structures,
        n_samples= Int(len(final_structures)),
        )

    qe_task = wg.add_task(
        qe, 
        name="QE_workflow",
        test_file= split_task.outputs.test_file,
        train_file= split_task.outputs.train_file,
        valid_file= split_task.outputs.valid_file,
        **qe_inputs
        )

    training_files = wg.add_task(
        create_train_file, 
        test_file= qe_task.outputs.test_file,
        train_file= qe_task.outputs.train_file,
        valid_file= qe_task.outputs.valid_file,
        )

    train_task = wg.add_task(
        trainCalc,
        mlip_config=training_files.outputs.JanusConfigfile,
        code=calc_inputs["code"],
        foundation_model=calc_inputs["model"],
        metadata=calc_inputs["metadata"],
        fine_tune=True,
    )

Visualise the workgraph

In [ ]:
wg


In [ ]:
wg.run()

In [ ]:
wg.tasks.Train.outputs

If we want to get the training plot, we have to pull it from the remote folder.  

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


folder = wg.tasks.Train.outputs.remote_folder.value
picturePath = f"{os.getcwd()}/traingraph.png"
folder.getfile(relpath='results/test_run-123_train_Default_stage_one.png',destpath=picturePath)

img = mpimg.imread(picturePath)
plt.imshow(img)